# 3-Qubit Entangled Circuit on IonQ Aria QPU

This notebook demonstrates how to create and submit a 3-qubit entangled circuit to IonQ Aria QPU via qBraid.
We'll create a GHZ (Greenberger-Horne-Zeilinger) state: |000⟩ + |111⟩

**Cost Estimates:**
- Basic run (1000 shots): 3,030 credits
- Error mitigation (2500 shots): 7,530 credits

## Import Required Libraries

In [ ]:
import qbraid
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import matplotlib.pyplot as plt

## Configuration

Set this to `True` to enable error mitigation (requires 2500 shots minimum)

In [ ]:
# Configuration
USE_ERROR_MITIGATION = False  # Change to True for error mitigation

if USE_ERROR_MITIGATION:
    shots = 2500  # Minimum for error mitigation
    print("Configuration: Error mitigation enabled (2500 shots)")
else:
    shots = 1000  # Basic run
    print("Configuration: Basic run (1000 shots)")

cost_credits = 3 * shots + 30  # 3 credits per shot + 30 per task
print(f"Estimated cost: {cost_credits} credits")

## Create 3-Qubit Entangled Circuit

We'll create a GHZ state using:
1. Hadamard gate on the first qubit to create superposition
2. CNOT gates to entangle all three qubits

In [ ]:
def create_3qubit_entangled_circuit():
    """Create a 3-qubit GHZ entangled state circuit"""
    # Create quantum and classical registers
    qreg = QuantumRegister(3, 'q')
    creg = ClassicalRegister(3, 'c')
    circuit = QuantumCircuit(qreg, creg)
    
    # Create GHZ state: |000⟩ + |111⟩
    circuit.h(qreg[0])      # Put first qubit in superposition
    circuit.cx(qreg[0], qreg[1])  # Entangle first and second qubits
    circuit.cx(qreg[1], qreg[2])  # Entangle second and third qubits
    
    # Measure all qubits
    circuit.measure(qreg, creg)
    
    return circuit

# Create the circuit
circuit = create_3qubit_entangled_circuit()
print("Created 3-qubit GHZ entangled circuit:")
print(circuit)

## Visualize the Circuit

In [ ]:
# Draw the circuit
circuit.draw(output='mpl')
plt.show()

## Connect to IonQ Aria Device

In [ ]:
# Get IonQ Aria device (try both Aria-1 and Aria-2)
try:
    device = qbraid.get_device("ionq_aria_1")
    device_name = "IonQ Aria-1"
except:
    try:
        device = qbraid.get_device("ionq_aria_2")
        device_name = "IonQ Aria-2"
    except Exception as e:
        print(f"Error connecting to Aria devices: {e}")
        device = None
        device_name = None

if device:
    print(f"Device: {device_name}")
    print(f"Device status: {device.status()}")
else:
    print("Could not connect to any Aria device")

## Submit Job to IonQ Aria

In [ ]:
if device:
    # Submit job
    job = device.run(circuit, shots=shots)
    print(f"Job submitted to {device_name} with {shots} shots")
    print(f"Job ID: {job.id}")
    
    if USE_ERROR_MITIGATION:
        print("Note: Using 2500 shots (minimum for error mitigation on Aria)")
    else:
        print("Note: Basic run without error mitigation")
    
    print(f"Estimated cost: {cost_credits} credits")
else:
    print("Cannot submit job - no device available")
    job = None

## Wait for Results and Display

In [ ]:
if job:
    # Wait for completion and get results
    print("Waiting for job completion...")
    result = job.result()
    
    print(f"Job Status: {job.status()}")
    print("Results:")
    counts = result.measurement_counts()
    print(f"Counts: {counts}")
else:
    print("No job to wait for")
    counts = None

## Visualize Results

In [ ]:
if counts:
    # Plot the results
    from qiskit.visualization import plot_histogram
    
    mitigation_text = "with Error Mitigation" if USE_ERROR_MITIGATION else "Basic Run"
    title = f"{device_name}: 3-Qubit GHZ State Results ({mitigation_text})"
    
    plot_histogram(counts, title=title)
    plt.show()
    
    # Expected results for perfect GHZ state should show mainly |000⟩ and |111⟩
    print("\nExpected results for perfect GHZ state:")
    print("- High counts for |000⟩ and |111⟩")
    print("- Low/zero counts for other states |001⟩, |010⟩, |011⟩, |100⟩, |101⟩, |110⟩")
    
    if USE_ERROR_MITIGATION:
        print("\nWith error mitigation, you should see:")
        print("- Better fidelity (closer to ideal 50/50 split between |000⟩ and |111⟩)")
        print("- Reduced noise in unwanted states")
else:
    print("No results to visualize")

## Compare Error Mitigation vs Basic Run

If you want to compare both approaches, change the configuration above and re-run the notebook.

In [ ]:
print("To compare error mitigation vs basic run:")
print("1. Run this notebook with USE_ERROR_MITIGATION = False")
print("2. Save the results")
print("3. Run again with USE_ERROR_MITIGATION = True")
print("4. Compare the fidelity and noise levels")
print("\nCost difference:")
print(f"- Basic run (1000 shots): {3 * 1000 + 30} credits")
print(f"- Error mitigation (2500 shots): {3 * 2500 + 30} credits")
print(f"- Additional cost for error mitigation: {(3 * 2500 + 30) - (3 * 1000 + 30)} credits")

## Summary

This notebook demonstrated:
1. Creating a 3-qubit GHZ entangled state circuit
2. Submitting the circuit to IonQ Aria QPU via qBraid
3. Options for basic run vs error mitigation
4. Retrieving and visualizing the results

The GHZ state should produce measurements primarily in the |000⟩ and |111⟩ basis states, demonstrating quantum entanglement across all three qubits. Error mitigation can improve the fidelity at the cost of additional shots and credits.